In [4]:
import pandas as pd
import numpy as np

In [2]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

--2024-05-20 06:03:37--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:212d:fe00:b:20a5:b140:21, 2600:9000:212d:1600:b:20a5:b140:21, 2600:9000:212d:7400:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:212d:fe00:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet’

yellow_tripdata_202 100%[===================>]  45.46M  53.6MB/s    in 0.8s    

2024-05-20 06:03:38 (53.6 MB/s) - ‘yellow_tripdata_2023-01.parquet’ saved [47673370/47673370]



In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

--2024-05-20 06:04:01--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:212d:fe00:b:20a5:b140:21, 2600:9000:212d:1600:b:20a5:b140:21, 2600:9000:212d:7400:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:212d:fe00:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47748012 (46M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-02.parquet’

yellow_tripdata_202 100%[===================>]  45.54M  51.4MB/s    in 0.9s    

2024-05-20 06:04:02 (51.4 MB/s) - ‘yellow_tripdata_2023-02.parquet’ saved [47748012/47748012]



## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [5]:
df_jan = pd.read_parquet('yellow_tripdata_2023-01.parquet')

In [6]:
df_jan.shape[1]

19

Question 1: 19

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59


In [7]:
df_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [8]:
df_jan.columns = df_jan.columns.str.lower()

In [9]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime

In [10]:
df_jan['duration'] = df_jan['duration'].apply(lambda td: td.seconds / 60)

In [11]:
df_jan[['tpep_dropoff_datetime', 'tpep_pickup_datetime', 'duration']]

,tpep_dropoff_datetime,tpep_pickup_datetime,duration
0,2023-01-01 00:40:36,2023-01-01 00:32:10,8.433333
1,2023-01-01 01:01:27,2023-01-01 00:55:08,6.316667
2,2023-01-01 00:37:49,2023-01-01 00:25:04,12.750000
3,2023-01-01 00:13:25,2023-01-01 00:03:48,9.616667
4,2023-01-01 00:21:19,2023-01-01 00:10:29,10.833333
...,...,...,...
3066761,2023-02-01 00:12:33,2023-01-31 23:58:34,13.983333
3066762,2023-01-31 23:50:36,2023-01-31 23:31:09,19.450000
3066763,2023-01-31 23:25:36,2023-01-31 23:01:05,24.516667
3066764,2023-01-31 23:53:00,2023-01-31 23:40:00,13.000000


In [12]:
df_jan['duration'].describe()

count    3.066766e+06
mean     1.565303e+01
std      4.162919e+01
min      0.000000e+00
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.439800e+03
Name: duration, dtype: float64

In [13]:
np.std(df_jan.duration)

41.62918432247523

Question 2: closest is 42.59

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [14]:
starting_length = df_jan.shape[0]

In [15]:
starting_length

3066766

In [16]:
df_jan = df_jan.loc[(df_jan.duration >= 1) & (df_jan.duration <= 60)]

In [17]:
df_jan.shape[0] / starting_length

0.9812212604417814

Question 3: 98%

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [18]:
df_jan['pulocationid'] = df_jan['pulocationid'].astype(str)
df_jan['dolocationid'] = df_jan['dolocationid'].astype(str)

In [19]:
from sklearn.feature_extraction import DictVectorizer

In [20]:
categorical = ['pulocationid', 'dolocationid']

In [21]:
dicts_train = df_jan[categorical].to_dict(orient='records')

In [22]:
dv = DictVectorizer()

In [23]:
X_train = dv.fit_transform(dicts_train)

In [24]:
X_train.shape

(3009176, 515)

Question 4: closest is 515

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [32]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [26]:
lr = LinearRegression()

In [27]:
# get the target variable
target = 'duration'
y_train = df_jan[target]

In [28]:
X_train.shape

(3009176, 515)

In [29]:
lr.fit(X_train, y_train)

LinearRegression()

In [30]:
y_pred = lr.predict(X_train)

In [33]:
rmse = root_mean_squared_error(y_train, y_pred)
rmse

7.64926220295883

Question 5: 7.64

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81


In [41]:
df_feb = pd.read_parquet('yellow_tripdata_2023-02.parquet')

In [40]:
def process_df(df):
    
    df = df.copy()
    df.columns = df.columns.str.lower()

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df['duration'].apply(lambda td: td.seconds / 60)

    df = df.loc[(df.duration >= 1) & (df.duration <= 60)]

    df['pulocationid'] = df['pulocationid'].astype(str)
    df['dolocationid'] = df['dolocationid'].astype(str)

    return df

In [42]:
df_feb = process_df(df_feb)

In [47]:
dicts_val = df_feb[categorical].to_dict(orient='records')

In [48]:
X_val = dv.transform(dicts_val)

In [49]:
y_pred = lr.predict(X_val)

In [51]:
y_val = df_feb['duration']

In [52]:
val_rmse = root_mean_squared_error(y_val, y_pred)
val_rmse

7.811797224429248

Question 6: 7.81